# Condor cluster for tW scattering

## Where to run?

This notebook was tested on uaf-7 and uaf-10.

```
( conda activate daskanalysisenv && jupyter notebook --no-browser )
```

## Troubleshooting

- Empty logs: why are the .out and .err logs empty most of the time?
- Why does a git clone of the analysis code not work on the workers? --> Because git is not installed on the workers. I need to manually ship the code.

```
condor_tail 11194580.0 -maxbytes 100000 -stderr
```

In [1]:
import os

from dask.distributed import Client
import distributed

from Tools.condor_utils import make_htcondor_cluster

from dask.distributed import Client, progress


In [2]:
cluster = make_htcondor_cluster(local=False, dashboard_address=13349, disk = "4GB", memory = "3GB",)
cluster

In [3]:
print ("Scaling cluster at address %s now."%cluster.scheduler_address)

cluster.scale(50)

os.environ["DASKSCHEDULER"] = str(cluster.scheduler_address)

Scaling cluster at address tcp://169.228.130.74:13753 now.


In [7]:
os.environ["DASKSCHEDULER"]

'tcp://169.228.130.74:13753'

In [5]:
c = Client(cluster)
c

Client Scheduler: tcp://169.228.130.74:13856 Dashboard: http://169.228.130.74:13349/status,Cluster Workers: 4 Cores: 4 Memory: 12.00 GB


In [4]:
if True:
    cluster.close()

In [10]:
cluster.get_logs()

{'Cluster': '',
 'Scheduler': "distributed.scheduler - INFO - Clear task state\ndistributed.scheduler - INFO -   Scheduler at: tcp://169.228.130.74:13856\ndistributed.scheduler - INFO -   dashboard at:                    :13349\ndistributed.scheduler - INFO - Register worker <Worker 'tcp://169.228.131.66:40164', name: 4, memory: 30, processing: 20>\ndistributed.scheduler - INFO - Starting worker compute stream, tcp://169.228.131.66:40164\ndistributed.scheduler - INFO - Register worker <Worker 'tcp://169.228.132.170:16280', name: 2, memory: 45, processing: 18>\ndistributed.scheduler - INFO - Starting worker compute stream, tcp://169.228.132.170:16280\ndistributed.scheduler - INFO - Register worker <Worker 'tcp://169.228.131.106:43724', name: 3, memory: 43, processing: 12>\ndistributed.scheduler - INFO - Starting worker compute stream, tcp://169.228.131.106:43724\ndistributed.scheduler - INFO - Register worker <Worker 'tcp://169.228.131.232:1498', name: 0, memory: 15, processing: 24>\ndistributed.scheduler - INFO - Starting worker compute stream, tcp://169.228.131.232:1498\ndistributed.scheduler - INFO - Receive client connection: Client-ad871405-2dc8-11eb-bb5f-ec0d9a0b03a0\ndistributed.scheduler - INFO - Receive client connection: Client-c4d043a1-2dc8-11eb-817f-bcaec51a4c96",
 'tcp://169.228.131.106:43724': 'distributed.worker - INFO -       Start worker at: tcp://169.228.131.106:43724\ndistributed.worker - INFO -          Listening to: tcp://169.228.131.106:43724\ndistributed.worker - INFO -          dashboard at:      169.228.131.106:35925\ndistributed.worker - INFO - Waiting to connect to: tcp://169.228.130.74:13856\ndistributed.worker - INFO - -------------------------------------------------\ndistributed.worker - INFO -               Threads:                          1\ndistributed.worker - INFO -                Memory:                    3.00 GB\ndistributed.worker - INFO -       Local Directory: /srv/temp/dask-worker-space/dask-worker-space/worker-5a9awxue\ndistributed.worker - INFO - -------------------------------------------------\ndistributed.worker - INFO -         Registered to: tcp://169.228.130.74:13856\ndistributed.worker - INFO - -------------------------------------------------',
 'tcp://169.228.131.232:1498': 'distributed.worker - INFO -       Start worker at: tcp://169.228.131.232:1498\ndistributed.worker - INFO -          Listening to: tcp://169.228.131.232:1498\ndistributed.worker - INFO -          dashboard at:      169.228.131.232:16793\ndistributed.worker - INFO - Waiting to connect to: tcp://169.228.130.74:13856\ndistributed.worker - INFO - -------------------------------------------------\ndistributed.worker - INFO -               Threads:                          1\ndistributed.worker - INFO -                Memory:                    3.00 GB\ndistributed.worker - INFO -       Local Directory: /srv/temp/dask-worker-space/dask-worker-space/worker-z4ofxi60\ndistributed.worker - INFO - -------------------------------------------------\ndistributed.worker - INFO -         Registered to: tcp://169.228.130.74:13856\ndistributed.worker - INFO - -------------------------------------------------',
 'tcp://169.228.131.66:40164': 'distributed.worker - INFO -       Start worker at: tcp://169.228.131.66:40164\ndistributed.worker - INFO -          Listening to: tcp://169.228.131.66:40164\ndistributed.worker - INFO -          dashboard at:       169.228.131.66:35410\ndistributed.worker - INFO - Waiting to connect to: tcp://169.228.130.74:13856\ndistributed.worker - INFO - -------------------------------------------------\ndistributed.worker - INFO -               Threads:                          1\ndistributed.worker - INFO -                Memory:                    3.00 GB\ndistributed.worker - INFO -       Local Directory: /srv/temp/dask-worker-space/dask-worker-space/worker-aqh9h6iw\ndistributed.worker - INFO - -------------------------------------------------\ndistributed.worker - INFO -         Registered to

In [6]:
# just to test 
def test(x):
    from Tools.cutflow import Cutflow
    return x**2

In [7]:
for res in map(test, range(5)):
    print (res)

0
1
4
9
16


In [8]:
import time
import random
futures = c.map(test, range(50))
results = c.gather(futures)

In [9]:
results

[0,
 1,
 4,
 9,
 16,
 25,
 36,
 49,
 64,
 81,
 100,
 121,
 144,
 169,
 196,
 225,
 256,
 289,
 324,
 361,
 400,
 441,
 484,
 529,
 576,
 625,
 676,
 729,
 784,
 841,
 900,
 961,
 1024,
 1089,
 1156,
 1225,
 1296,
 1369,
 1444,
 1521,
 1600,
 1681,
 1764,
 1849,
 1936,
 2025,
 2116,
 2209,
 2304,
 2401]

In [4]:
from tqdm import tqdm
for i in tqdm(range(10000)):
    x = i**2

100%|██████████| 10000/10000 [00:00<00:00, 480001.83it/s]
